In [48]:
import numpy as np
import matplotlib.pyplot as plt
from microboone_utils import *
import pandas as pd
from h5py import File

In [49]:
#abre o arquivo
f = File('/home/gabriel/Downloads/bnb_WithWire_00.h5','r')

In [50]:
#mostra todas as tabelas disponiveis no arquivo .h5 aberto

labels=[]
sub_labels=[]
for lines in f:
        labels.append(lines)
        aux=[]
        for key in f[lines]:
            aux.append(key)
        sub_labels.append(aux)
labels

['edep_table',
 'event_table',
 'hit_table',
 'opflash_table',
 'opflashsumpe_table',
 'ophit_table',
 'pandoraHit_table',
 'pandoraPfp_table',
 'pandoraPrimary_table',
 'particle_table',
 'wire_table']

In [51]:
#montar uma tavela dos eventos disponiveis na file
size=len(f[labels[1]]["event_id"][:])

names_event=["run","subrun","event"]
names_event_values=[[] for _ in range(len(names_event))]
for i in range(size):
    for j in range(len(names_event)):
        names_event_values[j].append(f[labels[1]]["event_id"][i][j])

temp_dict = {}
for i, name in enumerate(names_event):
    temp_dict[name] = names_event_values[i]


event_table_pd = pd.DataFrame(temp_dict)
names_event_values.clear()    
event_table_pd

,run,subrun,event
0,7004,331,16551
1,7004,331,16552
2,7004,331,16553
3,7004,331,16554
4,7004,331,16555
...,...,...,...
1080,7020,1439,72000
1081,5078,78,3904
1082,5078,78,3916
1083,5078,78,3941


In [52]:
my_index=0
this_run=event_table_pd["run"][my_index]
this_subrun=event_table_pd["subrun"][my_index]
this_event=event_table_pd["event"][my_index]

evt_id = [this_run, this_subrun, this_event]

In [55]:
wire_table=f[labels[10]]
event_wire_plan=wire_table["event_id"][:]
names_event=["run","subrun","event"]
names_event_values=[[] for _ in range(len(names_event))]
for i in range(max):
    for j in range(len(names_event)):
        names_event_values[j].append(wire_table["event_id"][i][j])
temp_dict = {}
for i, name in enumerate(names_event):
    temp_dict[name] = names_event_values[i]
temp_table=pd.DataFrame(temp_dict)

In [56]:
temp_table

,run,subrun,event
0,7004,331,16551
1,7004,331,16551
2,7004,331,16551
3,7004,331,16551
4,7004,331,16551
...,...,...,...
19995,7004,331,16553
19996,7004,331,16553
19997,7004,331,16553
19998,7004,331,16553


In [60]:
temp_table.index[(temp_table["run"]==evt_id[0]) & (temp_table["subrun"]==evt_id[1]) & (temp_table["event"]==evt_id[2]) ]

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       8246, 8247, 8248, 8249, 8250, 8251, 8252, 8253, 8254, 8255],
      dtype='int64', length=8256)

In [20]:
#montar a tabela de wire

max=20000 #tentar ver depois como pegar evento por evento melhor!!!!

wire_table=f[labels[10]]
event_wire_plan=wire_table["event_id"][0:max]
local_plane = wire_table["local_plane"][0:max]
adc=wire_table["adc"][0:max]
adc = [adc.tolist() for adc in adc]

names_event=["run","subrun","event"]
names_event_values=[[] for _ in range(len(names_event))]
for i in range(max):
    for j in range(len(names_event)):
        names_event_values[j].append(wire_table["event_id"][i][j])

temp_dict = {}
for i, name in enumerate(names_event):
    temp_dict[name] = names_event_values[i]

temp_dict["local_plane"]=local_plane.flatten()
temp_dict["adc"]=adc


wire_table_pd = pd.DataFrame(temp_dict)

adc.clear()
event_wire_plan=None
local_plane=None
wire_table=None

In [21]:
#vamos filtar por eventos, depois esse bloco tem que ser um for, junto com a parte de cima ... uhmmm
wire_table_pd=wire_table_pd.query("run==@this_run and subrun==@this_subrun and event==@this_event")

In [22]:
planeadcs=[]
for p in range(0,nplanes()):
    table_aux=wire_table_pd.query("local_plane==@p")
    aux=[]
    for j in range(len(table_aux)):
        aux.append(table_aux.iloc(0)[j]["adc"])
    planeadcs.append(np.array(aux))

wire_table_pd=wire_table_pd.drop(columns="adc")

In [23]:
from skimage.measure import block_reduce
f_downsample = 6
for p in range(0,nplanes()):
    planeadcs[p] = block_reduce(planeadcs[p], block_size=(1,f_downsample), func=np.sum)

adccutoff = 10.*f_downsample/6.
adcsaturation = 100.*f_downsample/6.
for p in range(0,nplanes()):
    planeadcs[p][planeadcs[p]<adccutoff] = 0
    planeadcs[p][planeadcs[p]>adcsaturation] = adcsaturation

In [29]:
zmax = adcsaturation

print("Run / Sub / Event : %i / %i / %i - saturation set to ADC sum=%.2f" % (evt_id[0], evt_id[1], evt_id[2], zmax))

# Plota a primeira imagem
plt.imshow(planeadcs[0].T, vmin=0, vmax=zmax, origin='lower', cmap='jet')
# Salva a primeira imagem
plt.tight_layout()
plt.savefig(f"./figuras/event_{evt_id[0]}_{evt_id[1]}_{evt_id[2]}_plane_0.png", bbox_inches='tight')
plt.clf()  # Limpa a figura para a próxima imagem

# Plota a segunda imagem
plt.imshow(planeadcs[1].T, vmin=0, vmax=zmax, origin='lower', cmap='jet')
# Salva a segunda imagem
plt.tight_layout()
plt.savefig(f"./figuras/event_{evt_id[0]}_{evt_id[1]}_{evt_id[2]}_plane_1.png", bbox_inches='tight')
plt.clf()  # Limpa a figura para a próxima imagem

# Plota a terceira imagem
plt.imshow(planeadcs[2].T, vmin=0, vmax=zmax, origin='lower', cmap='jet')
# Salva a terceira imagem
plt.tight_layout()
plt.savefig(f"./figuras/event_{evt_id[0]}_{evt_id[1]}_{evt_id[2]}_plane_2.png", bbox_inches='tight')
plt.clf()  # Limpa a figura para a próxima imagem


Run / Sub / Event : 7004 / 331 / 16552 - saturation set to ADC sum=100.00


<Figure size 640x480 with 0 Axes>